In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, clean_data,reg_ex     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# parameters
customer_id = '90'
formatted_attribute = 'Diameter'
customer_name='%WBMason%'
buckets = """Office Chairs"""

strategy_version_input=762
attribute_id_input=2730

In [3]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
# buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
# INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
# WHERE sba.attribute_id = {attribute_id_input}
# AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# # buckets

In [5]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

from fractions import Fraction
def frac2string_float(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    print(i)
    return str(round_string_float(float(i)))
# re.sub(r'([0-9]+(\.[0-9]+)?)', frac2string_float, x) 

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A



In [6]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
custom_field_df=pd.json_normalize(df['custom_fields'])
# custom_field_df = pd.json_normalize(df['custom_fields'])
fields = ['Width [Nom]', 'Width']
df_cur = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df_cur))

324


In [7]:
# print(len(df_custom))
# df_custom['match_cust']=df_custom['Diameter [Nom]'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'\.290','.29',str(x))).apply(lambda x: re.sub(r'\.450','.45',str(x))).apply(lambda x: re.sub(r'"','in',str(x))).apply(lambda x: re.sub(r'(?<! )in',' in',str(x))).apply(lambda x: re.sub(r'1500','1,500',str(x)))
                                                                       
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'''\[\'|\'\]|Up to ''','',str(x)))
# df_custom[curation_col]=df_custom['match_cust']
# regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\d+\.\d0)|(\,)|([a-eghj-mo-su-z])|(\.(?!\d))|(\d\d\d\d)|()'          
# df_custom['rounding'] = df_custom[curation_col].apply(lambda x: re_extract(regex_pattern_rounding, x))
# df_custom[df_custom['rounding'].astype(str)!='[]']

In [8]:
# matchcustom=df_custom[['external_id',curation_col]]
# matchcustom

In [9]:
# df = query_from_file(file_name='./query/uncurated.sql', params=params)
# df = df[df['bucket_id'].isin(buckets)]
# len(df)

In [10]:
# del df['product_id']
# del df['customer_id']
# del df['attributes']
# del df['bucket_id']
# del df['values']

In [11]:
# df[df['external_id'].astype(str)=='5637505279']
# df[df['buckets'].astype(str)=='Labels, Stickers & Tickets'][0:500]

In [12]:
# df['buckets'].explode().value_counts()

In [13]:
# def regex_patterns(unit):
#     return fr'(?i)(\d+\.\d+[\W]?[\W]?[\W]{unit}\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]{unit}\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]{unit}\b)|()'
# print(regex_patterns('Le?n?g?t?h?'))
# print(regex_patterns('He?i?g?h?t?'))
# print(regex_patterns('Wi?d?t?h?'))
# print(regex_patterns('Diam?e?t?e?r?'))

# Good Stuff

In [14]:
# regex_pattern_length=r'(?i)(\d+\.\d+.?.?.?Le?n?g?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Le?n?g?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Le?n?g?t?h?\b)|()'         
# df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
# regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
# df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
# regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
# df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df_cur['matches_diameter'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]|\d\'.?diameter|(\bround\b)|(circ)|(lamp)|(dot)"
df_cur['matches_diams'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]|(lamp)"
df_cur['matches_ft'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d'
df_cur['new_matches'] = df_cur['matches_diameter'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'(?i)(\d+.?\d*\/?\d*.?I?n?c?h?"?.?x.?\d+.?\d*\/?\d*.?I?n?c?h?"?)|(\d+.?ft)|(\d+.?foot)|(\d+.?feet)|(\d+.?inch)|(\d+.?")|(\d+")|()'
df_cur['blank_matches'] = df_cur['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df_cur['blank_matches_long'] = df_cur['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [15]:
height_new=df_cur[(df_cur['matches_diameter'].astype(str)!='[]')&(df_cur['matches_ft'].astype(str)=='[]')]
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: natsorted(x))
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: re.sub(r"\\?'.?diam?e?t?e?r?"," ft",str(x))).apply(lambda x: re.sub(r'(?i)\\?\".?diam?e?t?e?r?|\\?\”?\D?diam?e?t?e?r?',' in',str(x))).apply(lambda x: re.sub(r'\\" H|"H|\"xa0H|xa0H|"|�','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])

height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
# del height_new['matches_diameter']
# del height_new['matches_diameter']

height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
height_new['matches_diameter']=height_new['matches_diameter'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

In [16]:
height_new[curation_col]=height_new['matches_diameter']
height_new[curation_col]=height_new[curation_col].apply(lambda x: re.sub(r'22.25","in','22.25 in',str(x))).apply(lambda x: re.sub(r'6in in','6 in',str(x))).apply(lambda x: re.sub(r'26.75","in','26.75 in',str(x)))
height_new[curation_col]=height_new[curation_col].apply(lambda x: re.sub(r'875','88',str(x))).apply(lambda x: re.sub(r'375','38',str(x)))
matchesdiam=height_new[['external_id',curation_col]]
print(len(matchesdiam))
height_new.head(100)
height_new[curation_col].explode().value_counts()
diameter=height_new[['external_id',curation_col]]
diameter

8


external_id Q:diameter
28   5637585783  ["20 in"]
60   5637600670  ["36 in"]
61   5637600671  ["36 in"]
62   5637600674  ["36 in"]
237  5637595380  ["36 in"]
247  5637550712  ["16 in"]
269  5637424202  ["60 in"]
270  5637424203  ["60 in"]

# NA

In [23]:
df_na=df_cur[(df_cur['matches_diameter'].astype(str)=='[]')&(df_cur['matches_diams'].astype(str)=='[]')]
print(len(df_na))
df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]

296


# Length-Good Stuff

In [18]:
# from natsort import natsorted
# regex_pattern=r'(?i)((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})\d+\.\d+[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})\d+.?\d+\/\d+[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.\d)[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.)[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|()'           
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# print(len(df[df['matches'].astype(str)!='[]']))
# df['matches']=df['matches'].apply(lambda x: natsorted(x))

In [19]:
# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_new=df[(df['new_matches'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')]
# df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r'\\" L|"L|L|l',' in',str(x))).apply(lambda x: re.sub(r'\\" L|"L|\"xa0L|xa0L|"','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

# df_new['matches_length']=df_new['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# # df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: re.sub('\\\" L',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
# del df_new['matches_height']
# del df_new['matches_ft']
# del df_new['new_matches']
# del df_new['blank_matches']
# del df_new['blank_matches_long']
# del df_new['matches']

# df_new['matches_length']=df_new['matches_length'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))


In [20]:
# print(len(df_new))
# df_new[curation_col]=df_new['matches_length']
# matchesnew=df_new[['external_id',curation_col]]
# matchesnew

In [21]:
# print('Origninal: '+str(len(df)))
# print('Curated: '+str(len(diameter)+len(matchesna)+len(df_custom)))
# print('Remaning: '+str(len(df)-len(diameter)-len(matchesna)-len(df_custom)))

In [31]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchesna))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(diameter))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matchcustom))

matchesna
__


In [32]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [36]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchesna)

In [37]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, diameter)

In [29]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name

# def looks_good(customer, attribute, df, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
#     matches.to_csv(f'{drive_path}/{customer} - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 

In [30]:
# looks_good('WBMason', attribute, df, matchcustom)